In [ ]:
pip install pyxlsb
pip install sqlalchemy psycopg2-binary


# Connexion BDD

In [ ]:
%pip install -q "sqlalchemy>=2" psycopg2-binary

In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus

PG_HOST = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
PG_PORT = 5432
PG_DB   = "aerotec_datawarehouse"
PG_USER = "aerotec_datawarehouse_user"
PG_PASS = "LHTYZJ3aUDI8IeylbA1SZs9M9TsKQ4To"

conn_str = (
    f"postgresql+psycopg2://{PG_USER}:{quote_plus(PG_PASS)}@{PG_HOST}:{PG_PORT}/{PG_DB}"
)

# Astuces de stabilité pour Render :
# - sslmode=require (obligatoire)
# - connect_timeout : évite de bloquer si le réseau/host ne répond pas
# - keepalives_* : évite les coupures silencieuses d’idle
engine = create_engine(
    conn_str,
    connect_args={
        "sslmode": "require",
        "connect_timeout": 5,
        "keepalives": 1,
        "keepalives_idle": 30,
        "keepalives_interval": 10,
        "keepalives_count": 3,
    },
    pool_pre_ping=True,     # ping avant réutilisation du pool
    pool_recycle=1800,      # recycle connexions > 30 min
    pool_size=5,
    max_overflow=5,
)

# Test rapide de connectivité (à garder au début du main)
try:
    with engine.connect() as conn:
        ver = conn.execute(text("select version();")).scalar()
        print("[ok] Connecté à PostgreSQL Render")
        print(ver)
except OperationalError as e:
    print("[error] Connexion PostgreSQL échouée:", e)
    raise


[ok] Connecté à PostgreSQL Render
PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit


 # fichier qualité des trigrammes.xlsx

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import os
import pandas as pd
import numpy as np
import re, unicodedata, warnings

# --- DB: SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus

# ================== CONFIG ================== #
FILE_PATH = r"C:\globasoft\aerotech\fic\fichier qualité des trigrammes.xlsx"
SHOW_SAMPLE = True
TARGET_SCHEMA = "public"          # schéma cible
IF_EXISTS_MODE = "replace"        # "replace" ou "append"

# Forcer certains noms de colonnes en DATE/DATETIME (optionnel)
FORCE_DATE_COLS = {"date_d_attribution", "date_de_retrait"}  # snake_case attendu

# ================== ACCÈS DB ================== #
REQUIRED_KEYS = ("PG_HOST","PG_PORT","PG_DB","PG_USER","PG_PASS")

def _collect_pg_config():
    """Récupère les creds depuis le 1er bloc (globals) OU depuis l'env."""
    cfg = {}
    g = globals()
    # 1) variables déjà définies dans le 1er bloc ?
    for k in REQUIRED_KEYS:
        if k in g and g[k]:
            cfg[k] = g[k]
    # 2) sinon variables d'environnement ?
    for k in REQUIRED_KEYS:
        if k not in cfg or not cfg[k]:
            v = os.getenv(k)
            if v:
                cfg[k] = v

    # défauts pratiques si seul le host Render est utilisé
    if "PG_HOST" not in cfg or not cfg["PG_HOST"]:
        cfg["PG_HOST"] = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
    if "PG_PORT" not in cfg or not cfg["PG_PORT"]:
        cfg["PG_PORT"] = 5432
    else:
        cfg["PG_PORT"] = int(cfg["PG_PORT"])

    missing = [k for k in REQUIRED_KEYS if k not in cfg or cfg[k] in ("", None)]
    if missing:
        raise RuntimeError(
            "Paramètres DB manquants: "
            + ", ".join(missing)
            + ". Définis-les dans le 1er bloc (PG_*) ou via variables d'environnement."
        )
    return cfg

def get_engine():
    """Réutilise l'engine existant si présent. Sinon, en crée un avec SSL Render."""
    g = globals()
    if "engine" in g and g["engine"] is not None:
        return g["engine"]

    cfg = _collect_pg_config()
    conn_str = (
        f"postgresql+psycopg2://{cfg['PG_USER']}:{quote_plus(cfg['PG_PASS'])}"
        f"@{cfg['PG_HOST']}:{cfg['PG_PORT']}/{cfg['PG_DB']}"
    )
    eng = create_engine(
        conn_str,
        connect_args={
            "sslmode": "require",
            "connect_timeout": 5,
            "keepalives": 1,
            "keepalives_idle": 30,
            "keepalives_interval": 10,
            "keepalives_count": 3,
        },
        pool_pre_ping=True,
        pool_recycle=1800,
        pool_size=5,
        max_overflow=5,
    )
    return eng

# ================== UTILS ================== #
def strip_accents_lower(s):
    if s is None or (isinstance(s, float) and pd.isna(s)): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s):
    s = strip_accents_lower(s)
    s = re.sub(r"\s+", " ", s.strip())           # normalise espaces
    s = re.sub(r"[\s\.\-]+", "_", s)             # espaces/points/tirets -> _
    s = re.sub(r"[^a-z0-9_]", "_", s)            # autres -> _
    s = re.sub(r"_+", "_", s).strip("_") or "col"
    return s

def pg_ident(name: str, maxlen=63) -> str:
    """PostgreSQL ident max 63 chars."""
    return name[:maxlen]

def parse_number_like(s: pd.Series) -> pd.Series:
    # gère espace normal, insécable et fine insécable
    x = s.astype("string")\
         .str.replace("\u00A0", " ", regex=False)\
         .str.replace("\u202F", " ", regex=False)\
         .str.replace(" ", "", regex=False)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))", "", regex=True)\
         .str.replace(",", ".", regex=False)
    return pd.to_numeric(x, errors="coerce")

# --- header smarter ---
HEADER_KEYWORDS = (
    "date", "motif", "rédact", "redact", "trig", "nom", "prénom", "prenom",
    "personnel", "site", "retrait", "édition", "edition"
)

def choose_header_row(df: pd.DataFrame, scan=25) -> int:
    limit = min(len(df), scan)
    header_candidate_idx = None
    header_candidate_hits = -1
    for i in range(limit):
        row = df.iloc[i].astype(str)
        hits = 0
        for v in row:
            t = strip_accents_lower(v)
            if t and any(k in t for k in HEADER_KEYWORDS):
                hits += 1
        if hits >= 2 and hits > header_candidate_hits:
            header_candidate_hits = hits
            header_candidate_idx = i
    if header_candidate_idx is not None:
        return header_candidate_idx
    best, idx = -1, 0
    for i in range(limit):
        row = df.iloc[i].astype(str)
        non_empty = row.map(lambda x: x.strip()!="").sum()
        texty = row.map(lambda x: bool(re.search(r"[A-Za-zÀ-ÿ]", x))).sum()
        score = non_empty*2 + texty
        if score>best: best, idx = score, i
    return idx

def normalize_columns(header_vals):
    cols, seen = [], {}
    for v in header_vals:
        s = snake_id(v) if (v is not None and str(v).strip()!="") else "col"
        seen[s] = seen.get(s,0)+1
        cols.append(s if seen[s]==1 else f"{s}_{seen[s]}")
    return cols

def drop_empty_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=1, how="all")
    blank_cols = [c for c in df.columns
                  if df[c].isna().all() or df[c].astype(str).str.strip().eq("").all()]
    return df.drop(columns=blank_cols) if blank_cols else df

# --- inférence minimale robuste ---
MAP_TRUE  = {"true","vrai","oui","yes","1","y","o"}
MAP_FALSE = {"false","faux","non","no","0","n"}

DATE_TOKEN_RE = re.compile(
    r"(?:\d{4}[-/\.]\d{1,2}[-/\.]\d{1,2})|(?:\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4})|"
    r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|janv|févr|fevr|avr|mai|juin|juil|sept|oct|nov|d[ée]c)",
    re.I
)
ISO_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$")
COLNAME_DATE_HINTS = ("date", "dt", "heure", "time")

def infer_col(s: pd.Series):
    s = s.copy()
    ss = s.astype("string").str.strip()
    ss = ss.where(~ss.fillna("").eq("0"), pd.NA)  # éviter 0 -> 1970-01-01

    # 🔒 Forçage par nom de colonne (date_de_retrait, date_d_attribution) sans warning
    if snake_id(s.name) in FORCE_DATE_COLS:
        val = ss.fillna("")
        iso_mask = val.str.match(ISO_DATE_RE)
        if iso_mask.mean() >= 0.5:
            # Majoritairement ISO -> pas de warning
            dt = pd.to_datetime(ss, errors="coerce", dayfirst=False)
        else:
            # Essaie les deux et garde le meilleur parse
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
                dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
                dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
            dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1

        valid_dt = dt.dropna()
        has_time = (len(valid_dt) and (valid_dt.dt.time.astype(str) != "00:00:00").mean() > 0.2)
        return (dt, "DATETIME" if has_time else "DATE")

    # 0) ISO
    iso_mask = ss.fillna("").str.match(ISO_DATE_RE)
    if iso_mask.mean() >= 0.5:
        dt_iso = pd.to_datetime(ss.where(iso_mask), errors="coerce", dayfirst=False)
        ok = dt_iso.notna().mean() if len(ss) else 0.0
        if ok >= 0.5:
            valid_dt = dt_iso.dropna()
            has_time = (len(valid_dt) and (valid_dt.dt.time.astype(str) != "00:00:00").mean() > 0.2)
            return (dt_iso, "DATETIME" if has_time else "DATE")

    # 1) Excel serial
    as_num = pd.to_numeric(ss, errors="coerce")
    frac_num = as_num.notna().mean() if len(ss) else 0.0
    if frac_num >= 0.9:
        mask = as_num.between(60, 2950000)
        parsed = pd.to_datetime(as_num.where(mask), unit="D", origin="1899-12-30", errors="coerce")
        ok = parsed.notna().mean() if len(ss) else 0.0
        if ok >= 0.7:
            valid_dt = parsed.dropna()
            has_time = (len(valid_dt) and (valid_dt.dt.time.astype(str) != "00:00:00").mean() > 0.2)
            return (parsed, "DATETIME" if has_time else "DATE")

    # 2) tokens / nom de colonne (tolérant aux colonnes clairsemées)
    colname_hint = any(h in strip_accents_lower(s.name or "") for h in COLNAME_DATE_HINTS)
    date_token_ratio = ss.fillna("").str.contains(DATE_TOKEN_RE, na=False).mean()
    looks_datey = colname_hint or (date_token_ratio >= 0.30)
    if looks_datey:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
            dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
            dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
        dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1

        n_all = len(ss)
        n_parsed = int(dt.notna().sum())
        nonempty_mask = ss.fillna("").str.strip().ne("")
        n_nonempty = int(nonempty_mask.sum())

        ok_abs = (n_parsed / n_all) if n_all else 0.0
        ok_nonempty = (n_parsed / n_nonempty) if n_nonempty else 0.0

        if colname_hint:
            accept = (n_parsed >= 3 and ok_nonempty >= 0.80) or (n_parsed >= 2 and ok_abs >= 0.20)
        else:
            accept = ok_abs >= 0.70

        if accept:
            valid_dt = dt.dropna()
            has_time = (len(valid_dt) and (valid_dt.dt.time.astype(str) != "00:00:00").mean() > 0.2)
            return (dt, "DATETIME" if has_time else "DATE")

    # 3) bool strict
    mb = ss.str.lower().map(lambda x: True if x in MAP_TRUE else (False if x in MAP_FALSE else pd.NA))
    if (mb.notna().mean() if len(ss) else 0) >= 0.98:
        return mb.astype("boolean"), "BOOL"

    # 4) nombre
    nums = parse_number_like(ss)
    if (nums.notna().mean() if len(ss.dropna()) else 0) >= 0.85:
        nz = nums.dropna()
        if len(nz) and np.isclose(nz, np.round(nz)).all():
            return nums.round().astype("Int64"), "INT"
        return nums.astype("Float64"), "FLOAT"

    # 5) texte
    return ss.astype("string"), "STRING"

def pg_type(tag: str):
    return {
        "DATE": satypes.Date(),
        "DATETIME": satypes.DateTime(),  # timestamp without time zone
        "INT": satypes.Integer(),
        "FLOAT": satypes.Float(precision=53),
        "BOOL": satypes.Boolean(),
        "STRING": satypes.Text()
    }.get(tag, satypes.Text())

# ================== DB WRITE ================== #
def write_df_to_postgres(engine, df: pd.DataFrame, table_name: str, schema_tags: dict):
    # cast pandas → types sûrs
    for c, tag in schema_tags.items():
        if tag == "DATE":
            df[c] = pd.to_datetime(df[c], errors="coerce").dt.date
        elif tag == "DATETIME":
            df[c] = pd.to_datetime(df[c], errors="coerce")

    dtype_map = {c: pg_type(tag) for c, tag in schema_tags.items()}

    df.to_sql(
        name=table_name,
        con=engine,
        schema=TARGET_SCHEMA,
        if_exists=IF_EXISTS_MODE,   # "replace" ou "append"
        index=False,
        dtype=dtype_map,
        method="multi",
        chunksize=1000
    )

# ================== MAIN ================== #
def main():
    xlsx = Path(FILE_PATH)
    if not xlsx.exists():
        print(f"[error] Fichier introuvable: {xlsx}")
        return

    # Connexion DB (réutilise le 1er bloc ou env)
    try:
        engine = get_engine()
        with engine.connect() as conn:
            ver = conn.execute(text("select version();")).scalar()
            print("[ok] Connecté à PostgreSQL")
            print(ver)
    except (OperationalError, RuntimeError) as e:
        print("[error] Connexion PostgreSQL échouée:", e)
        return

    # Lecture multi-feuilles
    sheets = pd.read_excel(xlsx, sheet_name=None, engine="openpyxl", header=None)

    file_base = snake_id(xlsx.stem)
    done = 0

    for name, raw in sheets.items():
        raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)
        if raw.empty:
            continue

        h = choose_header_row(raw, scan=25)
        cols = normalize_columns(raw.iloc[h].tolist())
        df = raw.iloc[h+1:].copy()
        df.columns = cols

        df = df.dropna(how="all")
        df = drop_empty_columns(df)
        if df.empty:
            continue

        schema = {}
        for c in df.columns:
            casted, tag = infer_col(df[c])
            df[c] = casted
            schema[c] = tag

        # Nom de table: public.<file> si 1 feuille, sinon public.<file>__<sheet>
        sheet_base = snake_id(name)
        table_name = file_base if len(sheets) == 1 else f"{file_base}__{sheet_base}"
        table_name = pg_ident(table_name)

        print(f"\n--- Feuille: {name} -> {TARGET_SCHEMA}.{table_name} ---")
        print(f"rows={len(df)} | cols={df.shape[1]}")
        print("Colonnes:", ", ".join(df.columns.astype(str)))
        print("\nSchema détecté:")
        for c in df.columns:
            pgt = type(pg_type(schema[c])).__name__.replace("TypeEngine", "")
            print(f"  - {c}: {schema[c]} -> {pgt}")

        if SHOW_SAMPLE:
            with pd.option_context("display.max_columns", 80, "display.width", 200):
                print("\nSample (top 8):")
                print(df.head(8))

        # Écriture DB
        try:
            write_df_to_postgres(engine, df.copy(), table_name, schema)
            print(f"[ok] Inserted into {TARGET_SCHEMA}.{table_name}")
            done += 1
        except Exception as e:
            print(f"[error] Échec insertion {TARGET_SCHEMA}.{table_name}: {e}")

    print(f"\nDone. Feuilles analysées & importées: {done}")

if __name__ == "__main__":
    main()


[ok] Connecté à PostgreSQL
PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit

--- Feuille: PDG -> public.fichier_qualite_des_trigrammes__pdg ---
rows=20 | cols=4
Colonnes: edition, date, motif, redacteur

Schema détecté:
  - edition: INT -> Integer
  - date: DATE -> Date
  - motif: STRING -> Text
  - redacteur: STRING -> Text

Sample (top 8):
    edition       date                                              motif    redacteur
9         1 2024-06-10                                           Création  S. BELMONTE
10        2 2024-09-16                           Ajout nouveaux arrivants    Y. RAGEOT
11        3 2024-10-21                                    Ajout couturier    Y. RAGEOT
12        4 2024-10-28  Mise à jour des dates d'entrée 
Ajout des habi...    Y. RAGEOT
13        5 2024-12-16  Ajout nouvelle arrivante - 1 personne - Feriel...  F.BOULHABEL
14        6 2024-12-18                              Départ d

# Conditionsdepaiement.xlsx

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import os
import pandas as pd
import re, unicodedata
from datetime import datetime

# --- DB: SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus

# ========= CONFIG =========
FILE_PATH = r"C:\globasoft\aerotech\fic\Conditionsdepaiement.xlsx"
SHOW_SAMPLE = True

# DB cible
TARGET_SCHEMA   = "public"
IF_EXISTS_MODE  = "replace"      # "replace" ou "append"
TABLE_NAME      = None           # None => snake(file_stem) ; sinon "conditions_paiement"

# ========= LOG =========
def now(): return datetime.now().strftime("%H:%M:%S")
def log(msg): print(f"[{now()}] {msg}", flush=True)

# ========= ACCÈS DB =========
REQUIRED_KEYS = ("PG_HOST","PG_PORT","PG_DB","PG_USER","PG_PASS")

def _collect_pg_config():
    """Récupère creds depuis globals/env, avec defaults Render."""
    cfg, g = {}, globals()
    for k in REQUIRED_KEYS:
        if k in g and g[k]:
            cfg[k] = g[k]
    for k in REQUIRED_KEYS:
        if k not in cfg or not cfg[k]:
            v = os.getenv(k)
            if v:
                cfg[k] = v

    if not cfg.get("PG_HOST"):
        cfg["PG_HOST"] = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
    cfg["PG_PORT"] = int(cfg.get("PG_PORT", 5432))

    missing = [k for k in REQUIRED_KEYS if not cfg.get(k)]
    if missing:
        raise RuntimeError("Paramètres DB manquants: " + ", ".join(missing))
    return cfg

def get_engine():
    """Réutilise engine global si présent, sinon crée avec SSL Render."""
    g = globals()
    if "engine" in g and g["engine"] is not None:
        return g["engine"]
    cfg = _collect_pg_config()
    conn_str = (
        f"postgresql+psycopg2://{cfg['PG_USER']}:{quote_plus(cfg['PG_PASS'])}"
        f"@{cfg['PG_HOST']}:{cfg['PG_PORT']}/{cfg['PG_DB']}"
    )
    eng = create_engine(
        conn_str,
        connect_args={
            "sslmode": "require",
            "connect_timeout": 5,
            "keepalives": 1,
            "keepalives_idle": 30,
            "keepalives_interval": 10,
            "keepalives_count": 3,
        },
        pool_pre_ping=True,
        pool_recycle=1800,
        pool_size=5,
        max_overflow=5,
    )
    return eng

# ========= UTILS =========
def strip_accents_lower(s: str) -> str:
    if s is None: return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s: str) -> str:
    s = strip_accents_lower(s)
    s = re.sub(r"\s+", " ", s.strip())
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "col"

def parse_number_like(series: pd.Series) -> pd.Series:
    """Normalise nombres FR/US et convertit en nombre (ou NaN)."""
    x = series.astype("string") \
        .str.replace("\u00A0", " ", regex=False) \
        .str.replace("\u202F", " ", regex=False) \
        .str.replace(" ", "", regex=False)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))", "", regex=True)  # 1.234.567 -> 1234567
    x = x.str.replace(",", ".", regex=False)                           # virgule -> point
    return pd.to_numeric(x, errors="coerce")

def pick_numeric_dtype(s: pd.Series):
    """
    Détermine un type SQLAlchemy pour une colonne existante SANS créer de nouvelles colonnes.
    - si toute la colonne est Int64 -> Integer
    - sinon si float -> Float
    - sinon -> Text
    """
    if pd.api.types.is_integer_dtype(s):
        return satypes.Integer()
    if pd.api.types.is_float_dtype(s):
        return satypes.Float(precision=53)
    return satypes.Text()

# ========= ECRITURE DB =========
def dtype_map_for_postgres(df: pd.DataFrame) -> dict:
    """
    Mapping typé pour to_sql (PostgreSQL) uniquement sur colonnes existantes.
    """
    dtypes = {}
    for c in df.columns:
        dtypes[c] = pick_numeric_dtype(df[c]) if (pd.api.types.is_numeric_dtype(df[c])) else satypes.Text()
    return dtypes

def write_to_db(engine, df: pd.DataFrame, table_name: str):
    dtypes = dtype_map_for_postgres(df)
    df.to_sql(
        name=table_name,
        con=engine,
        schema=TARGET_SCHEMA,
        if_exists=IF_EXISTS_MODE,
        index=False,
        dtype=dtypes,
        method="multi",
        chunksize=1000
    )

# ========= MAIN =========
def main():
    xlsx = Path(FILE_PATH)
    log(f"Fichier: {xlsx}")
    if not xlsx.exists():
        log("[error] Fichier introuvable")
        return

    try:
        # header=None => pas d’en-têtes dans le fichier d’origine
        df0 = pd.read_excel(xlsx, sheet_name=0, header=None, engine="openpyxl")
    except Exception as e:
        log(f"[error] Lecture échouée: {e}")
        return

    log(f"Feuille[0]: shape={df0.shape}")

    # Purge lignes/colonnes totalement vides
    df = df0.dropna(how="all", axis=0).dropna(how="all", axis=1)
    log(f"Après drop vides: {df0.shape} -> {df.shape}")
    if df.empty:
        log("Feuille vide après nettoyage.")
        return

    # Respect strict du schéma du fichier : 2 ou 3 colonnes MAX, PAS d’ajout
    if df.shape[1] >= 3:
        df = df.iloc[:, :3].copy()
        # Noms techniques stables (puisqu’il n’y a pas d’en-têtes) — mais pas de nouvelle colonne
        df.columns = ["code_condition", "libelle", "delai_source"]
        log("Colonnes du fichier conservées: code_condition, libelle, delai_source")
    elif df.shape[1] == 2:
        df = df.iloc[:, :2].copy()
        df.columns = ["code_condition", "libelle"]
        log("Colonnes du fichier conservées: code_condition, libelle")
    else:
        log("[warn] Moins de 2 colonnes non vides trouvées. Abandon.")
        return

    # Nettoyage léger SANS créer de colonne : trims + tentative de parse numérique sur ce qui existe seulement
    if "libelle" in df.columns:
        df["libelle"] = df["libelle"].astype("string").str.strip()

    # Si le code est clairement numérique, on le convertit (sinon on le laisse tel quel)
    if "code_condition" in df.columns:
        s = df["code_condition"].astype("string").str.strip()
        non_na = s.dropna()
        all_digits = non_na.str.fullmatch(r"\d+").all() if len(non_na) else True
        if all_digits:
            df["code_condition"] = parse_number_like(df["code_condition"]).astype("Int64")

    # Si delai_source existe, on essaye juste un parse numérique (sans créer d’autre colonne)
    if "delai_source" in df.columns:
        parsed = parse_number_like(df["delai_source"])
        # Si tout est int (hors NaN), on caste en Int64 ; sinon on laisse tel quel
        if parsed.dropna().apply(float.is_integer).all():
            df["delai_source"] = parsed.astype("Int64")
        else:
            # si mixte, garde la série d’origine (respect total du fichier)
            pass

    # Affichage
    print("\n--- Conditions de paiement (collecte) ---")
    print(f"rows={len(df)} | cols={df.shape[1]}")
    print("Colonnes:", ", ".join(df.columns.astype(str)))

    print("\nSchéma final (identique au fichier, sans colonnes ajoutées):")
    for c in df.columns:
        print(f"  - {c}: dtype pandas = {df[c].dtype}")

    if SHOW_SAMPLE:
        with pd.option_context("display.max_columns", 120, "display.width", 220):
            print("\nSample (top 15):")
            print(df.head(15))

    # === Connexion & écriture DB ===
    file_base = snake_id(xlsx.stem)  # ex: "conditionsdepaiement"
    table_name = TABLE_NAME or file_base

    try:
        engine = get_engine()
        with engine.connect() as conn:
            ver = conn.execute(text("select version();")).scalar()
            log("[ok] Connecté PostgreSQL")
            log(ver)
    except (OperationalError, RuntimeError) as e:
        log(f"[error] Connexion PostgreSQL échouée: {e}")
        return df

    try:
        write_to_db(engine, df, table_name)
        log(f"[ok] Inserted into {TARGET_SCHEMA}.{table_name}")
    except Exception as e:
        log(f"[error] Échec insertion {TARGET_SCHEMA}.{table_name}: {e}")

    return df

if __name__ == "__main__":
    df_conditions = main()


[15:31:00] Fichier: C:\globasoft\aerotech\fic\Conditionsdepaiement.xlsx
[15:31:00] Feuille[0]: shape=(19, 3)
[15:31:00] Après drop vides: (19, 3) -> (19, 3)
[15:31:00] Colonnes du fichier conservées: code_condition, libelle, delai_source

--- Conditions de paiement (collecte) ---
rows=19 | cols=3
Colonnes: code_condition, libelle, delai_source

Schéma final (identique au fichier, sans colonnes ajoutées):
  - code_condition: dtype pandas = float64
  - libelle: dtype pandas = string
  - delai_source: dtype pandas = Int64

Sample (top 15):
    code_condition                        libelle  delai_source
0              1.0             Chèque à réception             0
1              6.0   Virement bancaire à 30 jours            30
2              7.0        Règlement à la commande             0
3              9.0              Chèque à 30 jours            30
4             15.0   Virement bancaire à 60 jours            60
5             16.0   Virement bancaire à 45 jours            45
6        

# FNP AEC projets - source.xlsx

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata, warnings
from datetime import datetime

# ============== CONFIG ==============
FILE_PATH = r"C:\globasoft\aerotech\fic\FNP AEC projets - source.xlsx"
SHOW_SAMPLE = True

# ============== LOG ==============
def now(): return datetime.now().strftime("%H:%M:%S")
def log(msg): print(f"[{now()}] {msg}", flush=True)

# ============== UTILS ==============
def strip_accents_lower(s):
    if s is None or (isinstance(s, float) and pd.isna(s)): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s):
    s = strip_accents_lower(s)
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "col"

def parse_number_like(series: pd.Series) -> pd.Series:
    x = series.astype("string").str.replace("\u00A0"," ", regex=False).str.replace(" ","", regex=False)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))","", regex=True)  # retire séparateurs de milliers
    x = x.str.replace(",",".", regex=False)                          # virgule -> point
    return pd.to_numeric(x, errors="coerce")

# ============== HEADER PICK + NORMALIZE ==============
HEADER_HINTS = (
    "projet","projets","code","id","date","mois","année","client","fournisseur","libelle","libellé",
    "montant","quantite","qté","qte","prix","ttc","ht","statut","status","site","type","categorie",
    "echeance","échéance","délai","delai","commentaire","ref","réf"
)

def choose_header_row(df: pd.DataFrame, scan=25) -> int:
    """1) ligne contenant plusieurs mots-clés; 2) densité + présence de texte."""
    limit = min(len(df), scan)
    cand_idx, cand_hits = None, -1
    for i in range(limit):
        row = df.iloc[i].astype(str)
        hits = 0
        for v in row:
            t = strip_accents_lower(v)
            if t and any(k in t for k in HEADER_HINTS):
                hits += 1
        if hits >= 2 and hits > cand_hits:
            cand_idx, cand_hits = i, hits
    if cand_idx is not None:
        return cand_idx

    best, idx = -1, 0
    for i in range(limit):
        row = df.iloc[i].astype(str)
        non_empty = row.map(lambda x: x.strip()!="").sum()
        texty = row.map(lambda x: bool(re.search(r"[A-Za-zÀ-ÿ]", x))).sum()
        score = non_empty*2 + texty
        if score > best: best, idx = score, i
    return idx

def normalize_columns(header_vals):
    cols, seen = [], {}
    for v in header_vals:
        s = snake_id(v) if (v is not None and str(v).strip()!="") else "col"
        seen[s] = seen.get(s,0)+1
        cols.append(s if seen[s]==1 else f"{s}_{seen[s]}")
    return cols

def drop_empty_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=1, how="all")
    blank_cols = [c for c in df.columns if df[c].dropna().astype(str).str.strip().eq("").all()]
    if blank_cols:
        df = df.drop(columns=blank_cols)
    return df

# ============== INFÉRENCE TYPES (simple et robuste) ==============
MAP_TRUE  = {"true","vrai","oui","yes","1","y","o"}
MAP_FALSE = {"false","faux","non","no","0","n"}

DATE_TOKEN_RE = re.compile(
    r"(?:\d{4}[-/\.]\d{1,2}[-/\.]\d{1,2})|(?:\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4})|"
    r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|janv|févr|fevr|avr|mai|juin|juil|sept|oct|nov|d[ée]c)",
    re.I
)
ISO_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$")
COLNAME_DATE_HINTS = ("date","dt","heure","time","echeance","échéance")

def excel_serial_to_datetime(series: pd.Series) -> pd.Series:
    """
    Conversion sûre des numéros de date Excel -> datetime.
    Fenêtre 'safe' contrainte à [60 .. 106750] jours:
    - 60  ~ 1900-03-01
    - 106750 ~ limite Timedelta pandas (~2173)
    Tout ce qui est hors-fenêtre -> NaN AVANT conversion.
    """
    vals = pd.to_numeric(series, errors="coerce").astype("float64")
    vals[~np.isfinite(vals)] = np.nan

    lower, upper = 60.0, 106750.0
    mask = (vals >= lower) & (vals <= upper)
    n_out = int((~mask & ~pd.isna(vals)).sum())
    if n_out:
        log(f"[excel_serial_to_datetime] valeurs hors fenêtre [{int(lower)}, {int(upper)}] remplacées par NaN: {n_out}")
    vals = vals.where(mask, np.nan)

    base = pd.Timestamp("1899-12-30")
    # conversion sûre (les NaN restent NaT)
    td = pd.to_timedelta(vals, unit="D", errors="coerce")
    dt = base + td
    return pd.to_datetime(dt, errors="coerce")

def infer_col(col: pd.Series):
    s = col.copy()
    ss = s.astype("string").str.strip()
    ss = ss.where(~ss.fillna("").eq("0"), pd.NA)  # éviter 0 -> 1970-01-01

    # 0) ISO direct
    iso_mask = ss.fillna("").str.match(ISO_DATE_RE)
    if iso_mask.mean() >= 0.5:
        dt_iso = pd.to_datetime(ss.where(iso_mask), errors="coerce", dayfirst=False)
        ok = dt_iso.notna().mean() if len(ss) else 0.0
        if ok >= 0.5:
            has_time = (dt_iso.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (
                dt_iso.dt.strftime("%Y-%m-%d %H:%M:%S") if has_time else dt_iso.dt.strftime("%Y-%m-%d"),
                "DATETIME" if has_time else "DATE"
            )

    # 1) Excel serial plausibles (et majoritairement dans la fenêtre sûre)
    as_num = pd.to_numeric(ss, errors="coerce")
    if len(ss):
        safe_mask = (as_num >= 60) & (as_num <= 106750)
        safe_ratio = safe_mask.mean()
    else:
        safe_ratio = 0.0

    if (as_num.notna().mean() if len(ss) else 0.0) >= 0.9 and safe_ratio >= 0.7:
        parsed = excel_serial_to_datetime(as_num)
        ok = parsed.notna().mean() if len(ss) else 0.0
        if ok >= 0.7:
            has_time = (parsed.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (
                parsed.dt.strftime("%Y-%m-%d %H:%M:%S") if has_time else parsed.dt.strftime("%Y-%m-%d"),
                "DATETIME" if has_time else "DATE"
            )

    # 2) tokens de date / nom “datey”
    colname_hint = any(h in strip_accents_lower(s.name or "") for h in COLNAME_DATE_HINTS)
    date_token_ratio = ss.fillna("").str.contains(DATE_TOKEN_RE, na=False).mean()
    looks_datey = colname_hint or (date_token_ratio >= 0.30)
    if looks_datey:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
            dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
            dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
        dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1
        ok = dt.notna().mean() if len(ss) else 0.0
        ok_thresh = 0.50 if colname_hint else 0.70
        if ok >= ok_thresh:
            has_time = (dt.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (
                pd.to_datetime(dt).dt.strftime("%Y-%m-%d %H:%M:%S") if has_time else pd.to_datetime(dt).dt.strftime("%Y-%m-%d"),
                "DATETIME" if has_time else "DATE"
            )

    # 3) bool strict
    mb = ss.str.lower().map(lambda x: True if x in MAP_TRUE else (False if x in MAP_FALSE else pd.NA))
    if (mb.notna().mean() if len(ss) else 0) >= 0.98:
        return mb.astype("boolean"), "BOOL"

    # 4) nombre
    nums = parse_number_like(ss)
    if (nums.notna().mean() if len(ss.dropna()) else 0) >= 0.85:
        nz = nums.dropna()
        if len(nz) and (np.mod(nz, 1) == 0).all():
            return nums.astype("Int64"), "INT"
        return nums.astype("Float64"), "FLOAT"

    # 5) texte
    return ss.astype("string"), "STRING"

def pg_type(tag: str) -> str:
    return {
        "DATE":"date","DATETIME":"timestamp without time zone",
        "INT":"integer","FLOAT":"double precision","BOOL":"boolean"
    }.get(tag, "text")

# ============== MAIN ==============
def main():
    xlsx = Path(FILE_PATH)
    log(f"Fichier: {xlsx}")
    if not xlsx.exists():
        log("[error] Fichier introuvable"); return

    try:
        sheets = pd.read_excel(xlsx, sheet_name=None, engine="openpyxl", header=None)
    except Exception as e:
        log(f"[error] Lecture échouée: {e}"); return

    done = 0
    for name, raw in sheets.items():
        log(f"\n[feuille] {name}: shape initiale={raw.shape}")

        # nettoyage vide
        raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)
        log(f"[feuille] {name}: après drop vides -> {raw.shape}")
        if raw.empty: 
            log(f"[feuille] {name}: vide, on passe.")
            continue

        # header + normalisation
        h = choose_header_row(raw, scan=25)
        log(f"[feuille] {name}: header choisi à la ligne {h}")
        cols = normalize_columns(raw.iloc[h].tolist())
        log(f"[feuille] {name}: colonnes normalisées -> {', '.join(cols)}")
        df = raw.iloc[h+1:].copy()
        df.columns = cols
        df = df.dropna(how="all")
        df = drop_empty_columns(df)
        log(f"[feuille] {name}: shape après nettoyage colonnes -> {df.shape}")
        if df.empty:
            log(f"[feuille] {name}: vide après normalisation, on passe.")
            continue

        # inférence types
        schema = {}
        for c in df.columns:
            casted, tag = infer_col(df[c])
            df[c] = casted
            schema[c] = tag

        base = f"{snake_id(xlsx.stem)}.{snake_id(name)}"
        print(f"\n--- Feuille: {name} -> {base} ---")
        print(f"rows={len(df)} | cols={df.shape[1]}")
        print("Colonnes:", ", ".join(df.columns.astype(str)))
        print("\nSchema détecté:")
        for c in df.columns:
            print(f"  - {c}: {schema[c]}  ->  {pg_type(schema[c])}")

        if SHOW_SAMPLE:
            with pd.option_context("display.max_columns", 120, "display.width", 220):
                print("\nSample (top 10):")
                print(df.head(10))

        done += 1

    print(f"\nDone. Feuilles analysées: {done}")

if __name__ == "__main__":
    main()


[15:24:20] Fichier: C:\globasoft\aerotech\fic\FNP AEC projets - source.xlsx
[15:24:21] 
[feuille] FNP achats: shape initiale=(1417, 23)
[15:24:21] [feuille] FNP achats: après drop vides -> (1417, 23)
[15:24:21] [feuille] FNP achats: header choisi à la ligne 0
[15:24:21] [feuille] FNP achats: colonnes normalisées -> periode, fournisseur_interne_externe, fournisseurs, code_projets, libelle_commande, type, n_cmd_aec, cdp, n_cmd_frs_date_cmd, montant_commande_ht, factures_recues, fact_recues, factures_non_recues, avancement_sur_factu_non_recues, montant_fnp, commentaires, compte_tiers, compte_charges, soumis_non_soumis, tva, reste, client_interne, section_analytique
[15:24:21] [feuille] FNP achats: shape après nettoyage colonnes -> (1416, 23)
[15:24:21] [excel_serial_to_datetime] valeurs hors fenêtre [60, 106750] remplacées par NaN: 149

--- Feuille: FNP achats -> fnp_aec_projets_source.fnp_achats ---
rows=1416 | cols=23
Colonnes: periode, fournisseur_interne_externe, fournisseurs, code_pr

# Projets AEC AE 2025 - source.xlsx

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import os
import pandas as pd
import numpy as np
import re, unicodedata, warnings
from datetime import datetime

# --- DB: SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus

# ============== CONFIG ==============
FILE_PATH = r"C:\globasoft\aerotech\fic\FNP AEC projets - source.xlsx"
SHOW_SAMPLE = True
TARGET_SCHEMA   = "public"
IF_EXISTS_MODE  = "replace"   # "replace" ou "append"

# ============== LOG ==============
def now(): return datetime.now().strftime("%H:%M:%S")
def log(msg): print(f"[{now()}] {msg}", flush=True)

# ============== ACCÈS DB ==============
REQUIRED_KEYS = ("PG_HOST","PG_PORT","PG_DB","PG_USER","PG_PASS")

def _collect_pg_config():
    cfg, g = {}, globals()
    for k in REQUIRED_KEYS:
        if k in g and g[k]:
            cfg[k] = g[k]
    for k in REQUIRED_KEYS:
        if k not in cfg or not cfg[k]:
            v = os.getenv(k)
            if v:
                cfg[k] = v
    if not cfg.get("PG_HOST"):
        cfg["PG_HOST"] = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
    cfg["PG_PORT"] = int(cfg.get("PG_PORT", 5432))
    missing = [k for k in REQUIRED_KEYS if not cfg.get(k)]
    if missing:
        raise RuntimeError("Paramètres DB manquants: " + ", ".join(missing))
    return cfg

def get_engine():
    g = globals()
    if "engine" in g and g["engine"] is not None:
        return g["engine"]
    cfg = _collect_pg_config()
    conn_str = (
        f"postgresql+psycopg2://{cfg['PG_USER']}:{quote_plus(cfg['PG_PASS'])}"
        f"@{cfg['PG_HOST']}:{cfg['PG_PORT']}/{cfg['PG_DB']}"
    )
    eng = create_engine(
        conn_str,
        connect_args={
            "sslmode": "require",
            "connect_timeout": 5,
            "keepalives": 1,
            "keepalives_idle": 30,
            "keepalives_interval": 10,
            "keepalives_count": 3,
        },
        pool_pre_ping=True,
        pool_recycle=1800,
        pool_size=5,
        max_overflow=5,
    )
    return eng

# ============== UTILS ==============
def strip_accents_lower(s):
    if s is None or (isinstance(s, float) and pd.isna(s)): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s):
    s = strip_accents_lower(s)
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "col"

def pg_ident(name: str, maxlen=63) -> str:
    return name[:maxlen]

def parse_number_like(series: pd.Series) -> pd.Series:
    x = series.astype("string").str.replace("\u00A0"," ", regex=False).str.replace(" ","", regex=False)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))","", regex=True)
    x = x.str.replace(",",".", regex=False)
    return pd.to_numeric(x, errors="coerce")

# ============== HEADER PICK + NORMALIZE ==============
HEADER_HINTS = (
    "projet","projets","code","id","date","mois","année","client","fournisseur","libelle","libellé",
    "montant","quantite","qté","qte","prix","ttc","ht","statut","status","site","type","categorie",
    "echeance","échéance","délai","delai","commentaire","ref","réf"
)

def choose_header_row(df: pd.DataFrame, scan=25) -> int:
    limit = min(len(df), scan)
    cand_idx, cand_hits = None, -1
    for i in range(limit):
        row = df.iloc[i].astype(str)
        hits = 0
        for v in row:
            t = strip_accents_lower(v)
            if t and any(k in t for k in HEADER_HINTS):
                hits += 1
        if hits >= 2 and hits > cand_hits:
            cand_idx, cand_hits = i, hits
    if cand_idx is not None:
        return cand_idx
    best, idx = -1, 0
    for i in range(limit):
        row = df.iloc[i].astype(str)
        non_empty = row.map(lambda x: x.strip()!="").sum()
        texty = row.map(lambda x: bool(re.search(r"[A-Za-zÀ-ÿ]", x))).sum()
        score = non_empty*2 + texty
        if score > best: best, idx = score, i
    return idx

def normalize_columns(header_vals):
    cols, seen = [], {}
    for v in header_vals:
        s = snake_id(v) if (v is not None and str(v).strip()!="") else "col"
        seen[s] = seen.get(s,0)+1
        cols.append(s if seen[s]==1 else f"{s}_{seen[s]}")
    return cols

def drop_empty_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=1, how="all")
    blank_cols = [c for c in df.columns if df[c].dropna().astype(str).str.strip().eq("").all()]
    if blank_cols:
        df = df.drop(columns=blank_cols)
    return df

# ============== INFÉRENCE TYPES ==============
MAP_TRUE  = {"true","vrai","oui","yes","1","y","o"}
MAP_FALSE = {"false","faux","non","no","0","n"}

DATE_TOKEN_RE = re.compile(
    r"(?:\d{4}[-/\.]\d{1,2}[-/\.]\d{1,2})|(?:\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4})|"
    r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|janv|févr|fevr|avr|mai|juin|juil|sept|oct|nov|d[ée]c)",
    re.I
)
ISO_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$")
COLNAME_DATE_HINTS = ("date","dt","heure","time","echeance","échéance")

def excel_serial_to_datetime(series: pd.Series) -> pd.Series:
    vals = pd.to_numeric(series, errors="coerce").astype("float64")
    vals[~np.isfinite(vals)] = np.nan
    lower, upper = 60.0, 106750.0
    mask = (vals >= lower) & (vals <= upper)
    vals = vals.where(mask, np.nan)
    base = pd.Timestamp("1899-12-30")
    td = pd.to_timedelta(vals, unit="D", errors="coerce")
    dt = base + td
    return pd.to_datetime(dt, errors="coerce")

def infer_col(col: pd.Series):
    """Retourne (serie_castée, TAG) avec vrais dtypes pandas, pas des strings formatées."""
    s = col.copy()
    ss = s.astype("string").str.strip()
    ss = ss.where(~ss.fillna("").eq("0"), pd.NA)

    # ISO
    iso_mask = ss.fillna("").str.match(ISO_DATE_RE)
    if iso_mask.mean() >= 0.5:
        dt_iso = pd.to_datetime(ss.where(iso_mask), errors="coerce", dayfirst=False)
        ok = dt_iso.notna().mean() if len(ss) else 0.0
        if ok >= 0.5:
            has_time = (dt_iso.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (dt_iso, "DATETIME" if has_time else "DATE")

    # Excel serial
    as_num = pd.to_numeric(ss, errors="coerce")
    if len(ss):
        safe_mask = (as_num >= 60) & (as_num <= 106750)
        safe_ratio = safe_mask.mean()
    else:
        safe_ratio = 0.0
    if (as_num.notna().mean() if len(ss) else 0.0) >= 0.9 and safe_ratio >= 0.7:
        parsed = excel_serial_to_datetime(as_num)
        ok = parsed.notna().mean() if len(ss) else 0.0
        if ok >= 0.7:
            has_time = (parsed.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (parsed, "DATETIME" if has_time else "DATE")

    # tokens / nom “datey”
    colname_hint = any(h in strip_accents_lower(s.name or "") for h in COLNAME_DATE_HINTS)
    date_token_ratio = ss.fillna("").str.contains(DATE_TOKEN_RE, na=False).mean()
    looks_datey = colname_hint or (date_token_ratio >= 0.30)
    if looks_datey:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
            dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
            dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
        dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1
        ok = dt.notna().mean() if len(ss) else 0.0
        ok_thresh = 0.50 if colname_hint else 0.70
        if ok >= ok_thresh:
            has_time = (dt.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (dt, "DATETIME" if has_time else "DATE")

    # bool
    mb = ss.str.lower().map(lambda x: True if x in MAP_TRUE else (False if x in MAP_FALSE else pd.NA))
    if (mb.notna().mean() if len(ss) else 0) >= 0.98:
        return mb.astype("boolean"), "BOOL"

    # nombre
    nums = parse_number_like(ss)
    if (nums.notna().mean() if len(ss.dropna()) else 0) >= 0.85:
        nz = nums.dropna()
        if len(nz) and (np.mod(nz, 1) == 0).all():
            return nums.astype("Int64"), "INT"
        return nums.astype("Float64"), "FLOAT"

    # texte
    return ss.astype("string"), "STRING"

def sa_type(tag: str):
    return {
        "DATE": satypes.Date(),
        "DATETIME": satypes.DateTime(),  # timestamp without time zone
        "INT": satypes.Integer(),
        "FLOAT": satypes.Float(precision=53),
        "BOOL": satypes.Boolean(),
        "STRING": satypes.Text()
    }.get(tag, satypes.Text())

def prepare_for_sql(df: pd.DataFrame, schema_tags: dict):
    out = df.copy()
    for c, tag in schema_tags.items():
        if tag == "DATE":
            out[c] = pd.to_datetime(out[c], errors="coerce").dt.date
        elif tag == "DATETIME":
            out[c] = pd.to_datetime(out[c], errors="coerce")
    dtype_map = {c: sa_type(tag) for c, tag in schema_tags.items()}
    return out, dtype_map

# ============== MAIN ==============
def main():
    xlsx = Path(FILE_PATH)
    log(f"Fichier: {xlsx}")
    if not xlsx.exists():
        log("[error] Fichier introuvable"); return

    try:
        sheets = pd.read_excel(xlsx, sheet_name=None, engine="openpyxl", header=None)
    except Exception as e:
        log(f"[error] Lecture échouée: {e}"); return

    # Connexion DB
    try:
        engine = get_engine()
        with engine.connect() as conn:
            ver = conn.execute(text("select version();")).scalar()
            log("[ok] Connecté PostgreSQL")
            log(ver)
    except (OperationalError, RuntimeError) as e:
        log(f"[error] Connexion PostgreSQL échouée: {e}")
        return

    file_base = snake_id(xlsx.stem)
    done = 0

    for name, raw in sheets.items():
        log(f"\n[feuille] {name}: shape initiale={raw.shape}")

        raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)
        log(f"[feuille] {name}: après drop vides -> {raw.shape}")
        if raw.empty:
            log(f"[feuille] {name}: vide, on passe."); continue

        h = choose_header_row(raw, scan=25)
        log(f"[feuille] {name}: header choisi à la ligne {h}")
        cols = normalize_columns(raw.iloc[h].tolist())
        log(f"[feuille] {name}: colonnes normalisées -> {', '.join(cols)}")

        df = raw.iloc[h+1:].copy()
        df.columns = cols
        df = df.dropna(how="all")
        df = drop_empty_columns(df)
        log(f"[feuille] {name}: shape après nettoyage colonnes -> {df.shape}")
        if df.empty:
            log(f"[feuille] {name}: vide après normalisation, on passe."); continue

        # Inférence types
        schema = {}
        for c in df.columns:
            casted, tag = infer_col(df[c])
            df[c] = casted
            schema[c] = tag

        # Nom de table: public.<file> si 1 feuille, sinon public.<file>__<sheet>
        sheet_base = snake_id(name)
        table_name = file_base if len(sheets) == 1 else f"{file_base}__{sheet_base}"
        table_name = pg_ident(table_name)

        # Affichage
        print(f"\n--- Feuille: {name} -> {TARGET_SCHEMA}.{table_name} ---")
        print(f"rows={len(df)} | cols={df.shape[1]}")
        print("Colonnes:", ", ".join(df.columns.astype(str)))
        print("\nSchema détecté:")
        for c in df.columns:
            print(f"  - {c}: {schema[c]} -> {sa_type(schema[c]).__class__.__name__}")

        if SHOW_SAMPLE:
            with pd.option_context("display.max_columns", 120, "display.width", 220):
                print("\nSample (top 10):")
                print(df.head(10))

        # Insertion DB
        try:
            df_sql, dtype_map = prepare_for_sql(df, schema)
            df_sql.to_sql(
                name=table_name,
                con=engine,
                schema=TARGET_SCHEMA,
                if_exists=IF_EXISTS_MODE,
                index=False,
                dtype=dtype_map,
                method="multi",
                chunksize=1000
            )
            log(f"[ok] Inserted into {TARGET_SCHEMA}.{table_name}")
            done += 1
        except Exception as e:
            log(f"[error] Échec insertion {TARGET_SCHEMA}.{table_name}: {e}")

    print(f"\nDone. Feuilles analysées & importées: {done}")

if __name__ == "__main__":
    main()


[15:45:14] Fichier: C:\globasoft\aerotech\fic\FNP AEC projets - source.xlsx
[15:45:15] [ok] Connecté PostgreSQL
[15:45:15] PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit
[15:45:15] 
[feuille] FNP achats: shape initiale=(1417, 23)
[15:45:15] [feuille] FNP achats: après drop vides -> (1417, 23)
[15:45:15] [feuille] FNP achats: header choisi à la ligne 0
[15:45:15] [feuille] FNP achats: colonnes normalisées -> periode, fournisseur_interne_externe, fournisseurs, code_projets, libelle_commande, type, n_cmd_aec, cdp, n_cmd_frs_date_cmd, montant_commande_ht, factures_recues, fact_recues, factures_non_recues, avancement_sur_factu_non_recues, montant_fnp, commentaires, compte_tiers, compte_charges, soumis_non_soumis, tva, reste, client_interne, section_analytique
[15:45:15] [feuille] FNP achats: shape après nettoyage colonnes -> (1416, 23)

--- Feuille: FNP achats -> public.fnp_aec_projets_source__fnp_achats ---
rows=14

# Relevé pointages AEB.xlsx

In [9]:
# -*- coding: utf-8 -*-

from pathlib import Path
import os
import pandas as pd
import numpy as np
import re, unicodedata, warnings
from datetime import datetime

# --- DB: SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus

# ============== CONFIG ==============
FILE_PATH = r"C:\globasoft\aerotech\fic\Relevé pointages AEB.xlsx"
SHOW_SAMPLE = True
STRICT_SOURCE_SCHEMA = True      # on suit EXACTEMENT le fichier (pas d'ajout/remap)
TARGET_SCHEMA = "public"
IF_EXISTS_MODE = "replace"       # "replace" ou "append"

# ============== LOG ==============
def now(): return datetime.now().strftime("%H:%M:%S")
def log(msg): print(f"[{now()}] {msg}", flush=True)

# ============== ACCÈS DB ==============
REQUIRED_KEYS = ("PG_HOST","PG_PORT","PG_DB","PG_USER","PG_PASS")

def _collect_pg_config():
    cfg, g = {}, globals()
    # 1) prendre éventuels globals (si définis dans un 1er bloc)
    for k in REQUIRED_KEYS:
        if k in g and g[k]:
            cfg[k] = g[k]
    # 2) compléter via variables d'environnement
    for k in REQUIRED_KEYS:
        if k not in cfg or not cfg[k]:
            v = os.getenv(k)
            if v:
                cfg[k] = v
    # défauts pratiques (Render)
    if not cfg.get("PG_HOST"):
        cfg["PG_HOST"] = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
    cfg["PG_PORT"] = int(cfg.get("PG_PORT", 5432))
    missing = [k for k in REQUIRED_KEYS if not cfg.get(k)]
    if missing:
        raise RuntimeError("Paramètres DB manquants: " + ", ".join(missing))
    return cfg

def get_engine():
    g = globals()
    if "engine" in g and g["engine"] is not None:
        return g["engine"]
    cfg = _collect_pg_config()
    conn_str = (
        f"postgresql+psycopg2://{cfg['PG_USER']}:{quote_plus(cfg['PG_PASS'])}"
        f"@{cfg['PG_HOST']}:{cfg['PG_PORT']}/{cfg['PG_DB']}"
    )
    eng = create_engine(
        conn_str,
        connect_args={
            "sslmode": "require",
            "connect_timeout": 5,
            "keepalives": 1,
            "keepalives_idle": 30,
            "keepalives_interval": 10,
            "keepalives_count": 3,
        },
        pool_pre_ping=True,
        pool_recycle=1800,
        pool_size=5,
        max_overflow=5,
    )
    return eng

# ============== UTILS ==============
def strip_accents_lower(s):
    if s is None or (isinstance(s, float) and pd.isna(s)): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s):
    s = strip_accents_lower(s)
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "col"

def make_unique_columns(cols):
    out, seen = [], {}
    for c in cols:
        if c not in seen:
            seen[c] = 1
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}_{seen[c]}")
    return out

def parse_number_like(series: pd.Series) -> pd.Series:
    x = series.astype("string").str.replace("\u00A0"," ", regex=False).str.replace(" ","", regex=False)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))","", regex=True)
    x = x.str.replace(",",".", regex=False)
    return pd.to_numeric(x, errors="coerce")

def drop_empty_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=1, how="all")
    blank_idx = []
    for j in range(df.shape[1]):
        s = df.iloc[:, j]
        if s.dropna().astype(str).str.strip().eq("").all():
            blank_idx.append(j)
    if blank_idx:
        log(f"[drop_empty_columns] suppression colonnes blanches (pos): {blank_idx}")
        df = df.drop(df.columns[blank_idx], axis=1)
    return df

# ============== HEADER PICK ==============
HEADER_HINTS = (
    # génériques
    "projet","projets","code","id","réf","ref","libelle","libellé","intitulé",
    "client","fournisseur","chef de projet","cp","cdp","manager",
    "statut","status","phase","categorie","catégorie","type",
    "date","heure","time","période","periode","echeance","échéance",
    "budget","montant","cout","coût","prix","ht","ttc","avancement","progress","%","taux",
    "site","section analytique","analytique","commentaire","observations","notes","remarques",
    # spécifiques vus dans le fichier de pointage
    "trigramme","nom","cat pointage","n° dossier","categorie","ot","ata","type de tâche","commentaires",
    "bu","immat","wp","semaine","mois","annee","année","cat point","annee - semaine","mois."
)

def choose_header_row(df: pd.DataFrame, scan=25) -> int:
    limit = min(len(df), scan)
    cand_idx, cand_hits = None, -1
    for i in range(limit):
        row = df.iloc[i].astype(str)
        hits = 0
        for v in row:
            t = strip_accents_lower(v)
            if t and any(k in t for k in HEADER_HINTS):
                hits += 1
        if hits >= 2 and hits > cand_hits:
            cand_idx, cand_hits = i, hits
    if cand_idx is not None:
        return cand_idx

    best, idx = -1, 0
    for i in range(limit):
        row = df.iloc[i].astype(str)
        non_empty = row.map(lambda x: x.strip()!="").sum()
        texty = row.map(lambda x: bool(re.search(r"[A-Za-zÀ-ÿ]", x))).sum()
        score = non_empty*2 + texty
        if score > best: best, idx = score, i
    return idx

# ============== INFÉRENCE TYPES (→ PostgreSQL) ==============
MAP_TRUE  = {"true","vrai","oui","yes","1","y","o"}
MAP_FALSE = {"false","faux","non","no","0","n"}

DATE_TOKEN_RE = re.compile(
    r"(?:\d{4}[-/\.]\d{1,2}[-/\.]\d{1,2})|(?:\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4})|"
    r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|janv|févr|fevr|avr|mai|juin|juil|sept|oct|nov|d[ée]c)",
    re.I
)
ISO_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$")
COLNAME_DATE_HINTS = ("date","dt","heure","time","echeance","échéance","debut","début","fin","période","periode","semaine","mois","année","annee")

AMOUNT_NAME_HINTS = (
    "montant","debit","débit","credit","crédit","solde","budget","ht","ttc",
    "amount","total","prix","coût","cout","heures","h"
)
CODE_LIKE_HINTS = ("code","nature","plan","compte","sop","section","niveau","trigramme","immat","ot","ata","wp","n° dossier","n_dossier","dossier","cat point","cat pointage")

def colname_has(hints, name):
    n = strip_accents_lower(name or "")
    return any(h in n for h in hints)

def excel_serial_to_datetime(series: pd.Series) -> pd.Series:
    vals = pd.to_numeric(series, errors="coerce").astype("float64")
    vals[~np.isfinite(vals)] = np.nan
    lower, upper = 60.0, 60000.0  # ~ 1900..2064
    mask = (vals >= lower) & (vals <= upper)
    n_out = int((~mask & ~pd.isna(vals)).sum())
    if n_out:
        log(f"[excel_serial_to_datetime] valeurs hors fenêtre [{int(lower)}, {int(upper)}] -> NaN: {n_out}")
    base = pd.Timestamp("1899-12-30")
    td = pd.to_timedelta(vals.where(mask, np.nan), unit="D", errors="coerce")
    dt = base + td
    return pd.to_datetime(dt, errors="coerce")

def infer_col(col: pd.Series):
    """
    Retourne (serie_casted, tag) avec tag ∈ {"DATE","DATETIME","INT","FLOAT","BOOL","STRING"}.
    On renvoie de VRAIS dtype pandas (datetime/int/float/bool/string), pas des strings formatées.
    """
    s = col.copy()
    ss = s.astype("string").str.strip()

    # 0) ISO direct
    iso_mask = ss.fillna("").str.match(ISO_DATE_RE)
    if len(ss) and iso_mask.mean() >= 0.5:
        dt_iso = pd.to_datetime(ss.where(iso_mask), errors="coerce", dayfirst=False)
        ok = dt_iso.notna().mean()
        if ok >= 0.5:
            has_time = (dt_iso.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (dt_iso, "DATETIME" if has_time else "DATE")

    # 1) Excel serial dates (restreint)
    as_num = pd.to_numeric(ss, errors="coerce")
    safe_mask = (as_num >= 60) & (as_num <= 60000)
    safe_ratio = safe_mask.mean() if len(ss) else 0.0

    colname_hint = colname_has(COLNAME_DATE_HINTS, s.name)
    high_vals_ratio = (as_num >= 20000).mean() if len(ss) else 0.0  # 20000 ~ 1954

    if (as_num.notna().mean() if len(ss) else 0.0) >= 0.9 and safe_ratio >= 0.7 and (colname_hint or high_vals_ratio >= 0.2):
        parsed = excel_serial_to_datetime(as_num)
        ok = parsed.notna().mean() if len(ss) else 0.0
        if ok >= 0.7:
            has_time = (parsed.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (parsed, "DATETIME" if has_time else "DATE")

    # 2) tokens de date / nom “datey”
    date_token_ratio = ss.fillna("").str.contains(DATE_TOKEN_RE, na=False).mean()
    looks_datey = colname_hint or (date_token_ratio >= 0.30)
    if looks_datey:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
            dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
            dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
        dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1
        ok = dt.notna().mean() if len(ss) else 0.0
        ok_thresh = 0.50 if colname_hint else 0.70
        if ok >= ok_thresh:
            has_time = (dt.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (dt, "DATETIME" if has_time else "DATE")

    # 3) bool strict
    mb = ss.str.lower().map(lambda x: True if x in MAP_TRUE else (False if x in MAP_FALSE else pd.NA))
    if len(ss) and mb.notna().mean() >= 0.98:
        return mb.astype("boolean"), "BOOL"

    # 3bis) colonnes monétaires / heures
    if colname_has(AMOUNT_NAME_HINTS, s.name):
        nums_hint = parse_number_like(ss)
        if nums_hint.notna().any():
            nz = nums_hint.dropna()
            if len(nz) and (np.mod(nz, 1) == 0).all():
                return nums_hint.astype("Int64"), "INT"
            return nums_hint.astype("Float64"), "FLOAT"

    # 3ter) colonnes "code-like" : garder du texte
    if colname_has(CODE_LIKE_HINTS, s.name):
        return ss.astype("string"), "STRING"

    # 4) nombre générique
    nums = parse_number_like(ss)
    if (nums.notna().mean() if len(ss) else 0) >= 0.85:
        nz = nums.dropna()
        if len(nz) and (np.mod(nz, 1) == 0).all():
            return nums.astype("Int64"), "INT"
        return nums.astype("Float64"), "FLOAT"

    # 5) texte
    return ss.astype("string"), "STRING"

def sa_type(tag: str):
    return {
        "DATE": satypes.Date(),
        "DATETIME": satypes.DateTime(),    # timestamp without time zone
        "INT": satypes.Integer(),
        "FLOAT": satypes.Float(precision=53),
        "BOOL": satypes.Boolean(),
        "STRING": satypes.Text()
    }.get(tag, satypes.Text())

def prepare_for_sql(df: pd.DataFrame, schema_tags: dict):
    out = df.copy()
    for c, tag in schema_tags.items():
        if tag == "DATE":
            out[c] = pd.to_datetime(out[c], errors="coerce").dt.date
        elif tag == "DATETIME":
            out[c] = pd.to_datetime(out[c], errors="coerce")
    dtype_map = {c: sa_type(tag) for c, tag in schema_tags.items()}
    return out, dtype_map

def pg_ident(name: str, maxlen=63) -> str:
    return name[:maxlen]

# ============== MAIN ==============
def main():
    xlsx = Path(FILE_PATH)
    log(f"Fichier: {xlsx}")
    if not xlsx.exists():
        log("[error] Fichier introuvable"); return

    # Connexion DB
    try:
        engine = get_engine()
        with engine.connect() as conn:
            ver = conn.execute(text("select version();")).scalar()
            log("[ok] Connecté PostgreSQL")
            log(ver)
    except (OperationalError, RuntimeError) as e:
        log(f"[error] Connexion PostgreSQL échouée: {e}")
        return

    try:
        sheets = pd.read_excel(xlsx, sheet_name=None, engine="openpyxl", header=None)
    except Exception as e:
        log(f"[error] Lecture échouée: {e}"); return

    file_base = snake_id(xlsx.stem)
    done = 0

    for name, raw in sheets.items():
        log(f"\n[feuille] {name}: shape initiale={raw.shape}")

        # nettoyage vide
        raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)
        log(f"[feuille] {name}: après drop vides -> {raw.shape}")
        if raw.empty:
            log(f"[feuille] {name}: vide, on passe.")
            continue

        # repère l'entête dans le brut (schéma source strict)
        h = choose_header_row(raw, scan=25)
        log(f"[feuille] {name}: header choisi à la ligne {h}")

        # ===== VUE SOURCE STRICTE =====
        df_src = raw.iloc[h+1:].copy()
        src_cols_original = raw.iloc[h].tolist()  # libellés EXACTS du fichier
        df_src.columns = src_cols_original
        df_src = df_src.dropna(how="all").copy()
        df_src = drop_empty_columns(df_src)
        log(f"[feuille] {name}: shape après en-têtes -> {df_src.shape}")
        if df_src.empty:
            log(f"[feuille] {name}: vide après normalisation, on passe.")
            continue

        # noms PostgreSQL correspondants (snake_case + unicité)
        pg_cols = make_unique_columns([snake_id(c) for c in df_src.columns])

        # inférence STRICTE sur colonnes d'origine
        schema_src = {}
        casted_df = df_src.copy()
        for c in list(df_src.columns):
            casted, tag = infer_col(df_src[c])
            casted_df[c] = casted
            schema_src[c] = tag

        # affichage (mapping & schéma)
        table_base = file_base if len(sheets) == 1 else f"{file_base}__{snake_id(name)}"
        table_name = pg_ident(table_base)

        print(f"\n--- Feuille: {name} -> {TARGET_SCHEMA}.{table_name} (SCHÉMA SOURCE STRICT) ---")
        print(f"rows={len(casted_df)} | cols={casted_df.shape[1]}")
        print("\nMapping colonnes (Excel → nom_pg) :")
        for src_name, pg_name in zip(df_src.columns, pg_cols):
            print(f"  - {src_name}  ->  {pg_name}")
        print("\nSchema détecté (types PostgreSQL) :")
        for src_name, pg_name in zip(df_src.columns, pg_cols):
            tag = schema_src.get(src_name, "STRING")
            print(f"  - {pg_name}: {sa_type(tag).__class__.__name__}  (source: {src_name}, tag: {tag})")

        if SHOW_SAMPLE:
            with pd.option_context("display.max_columns", 180, "display.width", 260):
                print("\nSample (top 10) :")
                print(casted_df.head(10))

        # ===== INSERTION DB (schéma strict) =====
        try:
            # renommer les colonnes pour la table Postgres
            df_for_sql = casted_df.copy()
            df_for_sql.columns = pg_cols

            # préparer dtypes & mapping
            schema_sql = {pg: schema_src[src] for src, pg in zip(df_src.columns, pg_cols)}
            df_sql, dtype_map = prepare_for_sql(df_for_sql, schema_sql)

            # to_sql
            df_sql.to_sql(
                name=table_name,
                con=engine,
                schema=TARGET_SCHEMA,
                if_exists=IF_EXISTS_MODE,
                index=False,
                dtype=dtype_map,
                method="multi",
                chunksize=1000
            )
            log(f"[ok] Inserted into {TARGET_SCHEMA}.{table_name}")
            done += 1
        except Exception as e:
            log(f"[error] Échec insertion {TARGET_SCHEMA}.{table_name}: {e}")

    print(f"\nDone. Feuilles analysées & importées: {done}")

if __name__ == "__main__":
    main()


[15:54:35] Fichier: C:\globasoft\aerotech\fic\Relevé pointages AEB.xlsx
[15:54:35] [ok] Connecté PostgreSQL
[15:54:35] PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit
[15:54:38] 
[feuille] Sheet1: shape initiale=(8980, 27)
[15:54:38] [feuille] Sheet1: après drop vides -> (8980, 27)
[15:54:38] [feuille] Sheet1: header choisi à la ligne 0
[15:54:38] [feuille] Sheet1: shape après en-têtes -> (8979, 27)

--- Feuille: Sheet1 -> public.releve_pointages_aeb (SCHÉMA SOURCE STRICT) ---
rows=8979 | cols=27

Mapping colonnes (Excel → nom_pg) :
  - Trigramme  ->  trigramme
  - NOM  ->  nom
  - DATE  ->  date
  - Cat Pointage  ->  cat_pointage
  - N° DOSSIER  ->  n_dossier
  - CATEGORIE  ->  categorie
  - OT  ->  ot
  - ATA  ->  ata
  - Type de tâche  ->  type_de_tache
  - Commentaires  ->  commentaires
  - Hrs  ->  hrs
  - Hrs supp  ->  hrs_supp
  - Heures Totals  ->  heures_totals
  - Société  ->  societe
  - SERVICE  ->  

# Suivi Projets AEC - source.xlsx

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import os
import pandas as pd
import numpy as np
import re, unicodedata, warnings
from datetime import datetime

# --- DB: SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy import types as satypes
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus

# ============== CONFIG ==============
FILE_PATH = r"C:\globasoft\aerotech\fic\Suivi Projets AEC - source.xlsx"
SHOW_SAMPLE = True
TARGET_SCHEMA = "public"
IF_EXISTS_MODE = "replace"  # "replace" ou "append"

# (optionnel) masquer un warning bruyant de pandas
warnings.filterwarnings("ignore", message=r"Parsing dates in %Y-%m-%d %H:%M:%S format.*", category=UserWarning)

# ============== LOG ==============
def now(): return datetime.now().strftime("%H:%M:%S")
def log(msg): print(f"[{now()}] {msg}", flush=True)

# ============== ACCÈS DB ==============
REQUIRED_KEYS = ("PG_HOST","PG_PORT","PG_DB","PG_USER","PG_PASS")

def _collect_pg_config():
    cfg, g = {}, globals()
    # 1) prendre éventuels globals (si définis dans un 1er bloc)
    for k in REQUIRED_KEYS:
        if k in g and g[k]:
            cfg[k] = g[k]
    # 2) compléter via variables d'environnement
    for k in REQUIRED_KEYS:
        if k not in cfg or not cfg[k]:
            v = os.getenv(k)
            if v:
                cfg[k] = v
    # défauts pratiques (Render)
    if not cfg.get("PG_HOST"):
        cfg["PG_HOST"] = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
    cfg["PG_PORT"] = int(cfg.get("PG_PORT", 5432))
    missing = [k for k in REQUIRED_KEYS if not cfg.get(k)]
    if missing:
        raise RuntimeError("Paramètres DB manquants: " + ", ".join(missing))
    return cfg

def get_engine():
    g = globals()
    if "engine" in g and g["engine"] is not None:
        return g["engine"]
    cfg = _collect_pg_config()
    conn_str = (
        f"postgresql+psycopg2://{cfg['PG_USER']}:{quote_plus(cfg['PG_PASS'])}"
        f"@{cfg['PG_HOST']}:{cfg['PG_PORT']}/{cfg['PG_DB']}"
    )
    eng = create_engine(
        conn_str,
        connect_args={
            "sslmode": "require",
            "connect_timeout": 5,
            "keepalives": 1,
            "keepalives_idle": 30,
            "keepalives_interval": 10,
            "keepalives_count": 3,
        },
        pool_pre_ping=True,
        pool_recycle=1800,
        pool_size=5,
        max_overflow=5,
    )
    return eng

# ============== UTILS ==============
def strip_accents_lower(s):
    if s is None or (isinstance(s, float) and pd.isna(s)): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s):
    s = strip_accents_lower(s)
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "col"

def make_unique_columns(cols):
    out, seen = [], {}
    for c in cols:
        if c not in seen:
            seen[c] = 1
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}_{seen[c]}")
    return out

def _preclean_numeric_strings(x: pd.Series) -> pd.Series:
    s = x.astype("string")
    s = s.str.replace(r"^(true|false|yes|no|vrai|faux)$", "", flags=re.I, regex=True)
    s = s.str.replace("\u00A0"," ", regex=False).str.replace(" ","", regex=False)
    s = s.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))","", regex=True)  # 1.234 -> 1234
    s = s.str.replace(",",".", regex=False)  # 1,23 -> 1.23
    return s

def parse_number_like(series: pd.Series) -> pd.Series:
    s = _preclean_numeric_strings(series)
    return pd.to_numeric(s, errors="coerce")

def drop_empty_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=1, how="all")
    blank_idx = []
    for j in range(df.shape[1]):
        s = df.iloc[:, j]
        if s.dropna().astype(str).str.strip().eq("").all():
            blank_idx.append(j)
    if blank_idx:
        log(f"[drop_empty_columns] suppression colonnes blanches (pos): {blank_idx}")
        df = df.drop(df.columns[blank_idx], axis=1)
    return df

# ============== HEADER PICK ==============
HEADER_HINTS = (
    "projet","projets","code","id","réf","ref","libelle","libellé","intitulé",
    "client","fournisseur","chef de projet","cp","cdp","manager",
    "statut","status","phase","categorie","catégorie","type",
    "date","heure","time","période","periode","echeance","échéance","fin","début","debut",
    "budget","montant","cout","coût","prix","ht","ttc","avancement","progress","%","taux",
    "site","section analytique","analytique","commentaire","observations","notes","remarques",
    "comments","revue raf","statut clipper","gtm","pôle","pole",
    "remaining forecast","facturation mois","hrs","heures","(h)",
    "ca ","charges","marge","achats","commandé","enregistré","fnp","int","raf"
)

def choose_header_row(df: pd.DataFrame, scan=30) -> int:
    limit = min(len(df), scan)
    cand_idx, cand_hits = None, -1
    for i in range(limit):
        row = df.iloc[i].astype(str)
        hits = 0
        for v in row:
            t = strip_accents_lower(v)
            if t and any(k in t for k in HEADER_HINTS):
                hits += 1
        if hits >= 2 and hits > cand_hits:
            cand_idx, cand_hits = i, hits
    if cand_idx is not None:
        return cand_idx
    best, idx = -1, 0
    for i in range(limit):
        row = df.iloc[i].astype(str)
        non_empty = row.map(lambda x: x.strip()!="").sum()
        texty = row.map(lambda x: bool(re.search(r"[A-Za-zÀ-ÿ]", x))).sum()
        score = non_empty*2 + texty
        if score > best: best, idx = score, i
    return idx

# ============== INFÉRENCE TYPES (→ PostgreSQL) ==============
MAP_TRUE  = {"true","vrai","oui","yes","1","y","o"}
MAP_FALSE = {"false","faux","non","no","0","n"}

DATE_TOKEN_RE = re.compile(
    r"(?:\d{4}[-/\.]\d{1,2}[-/\.]\d{1,2})|(?:\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4})|"
    r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|janv|févr|fevr|avr|mai|juin|juil|sept|oct|nov|d[ée]c)",
    re.I
)
ISO_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$")

DATE_NAME_HINTS   = ("date","dt","heure","time","echeance","échéance","debut","début","fin","période","periode","semaine","mois","année","annee")
MONEY_NAME_HINTS  = ("€","ht","budget","montant","amount","total","prix","coût","cout","facturation","forecast","achats","commandé","enregistré","fnp","ca ","charges","marge","int","raf")
HOUR_NAME_HINTS   = ("heures","hrs","(h)"," h ")
PERCENT_NAME_HINTS= ("%", "taux", "marge", "avancement", "écart", "ecart", "pourcent")
CODE_LIKE_HINTS   = ("code","nature","plan","compte","sop","section","niveau","trigramme","immat","ot","ata","wp","n° dossier","n_dossier","dossier","cat point","cat pointage","statut clipper","gtm","pôle","pole","comments","cdp","top custo","réunion raf","reunion raf","statut suivi de projets","statut suivi de projet")

def colname_has(hints, name):
    n = strip_accents_lower(name or "")
    return any(h in n for h in hints)

def excel_serial_to_datetime(series: pd.Series) -> pd.Series:
    vals = pd.to_numeric(series, errors="coerce").astype("float64")
    vals[~np.isfinite(vals)] = np.nan
    lower, upper = 60.0, 60000.0  # ~ 1900..2064
    mask = (vals >= lower) & (vals <= upper)
    n_out = int((~mask & ~pd.isna(vals)).sum())
    if n_out:
        log(f"[excel_serial_to_datetime] valeurs hors fenêtre [{int(lower)}, {int(upper)}] -> NaN: {n_out}")
    base = pd.Timestamp("1899-12-30")
    td = pd.to_timedelta(vals.where(mask, np.nan), unit="D", errors="coerce")
    dt = base + td
    return pd.to_datetime(dt, errors="coerce")

def infer_col(col: pd.Series):
    """
    Retourne (serie_casted, tag) avec tag ∈ {"DATE","DATETIME","INT","FLOAT","BOOL","STRING"}.
    PRÉCÉDENCE EN-TÊTE > CONTENU. Renvoie de VRAIS dtypes pandas (datetime/int/float/bool/string),
    pas de strings formatées pour les dates.
    """
    s = col.copy()
    ss = s.astype("string").str.strip()
    name = s.name or ""

    # 1) Colonnes monétaires / métriques -> float
    if colname_has(MONEY_NAME_HINTS, name):
        nums = parse_number_like(ss)
        if nums.notna().any():
            return nums.astype("Float64"), "FLOAT"

    # 2) Heures -> float
    if colname_has(HOUR_NAME_HINTS, name):
        nums = parse_number_like(ss)
        if nums.notna().any():
            return nums.astype("Float64"), "FLOAT"

    # 3) % / taux -> float
    if ("%" in (name or "")) or colname_has(PERCENT_NAME_HINTS, name):
        nums = parse_number_like(ss)
        if nums.notna().any():
            return nums.astype("Float64"), "FLOAT"

    # 4) Dates ISO explicites -> datetime/date (pandas)
    iso_mask = ss.fillna("").str.match(ISO_DATE_RE)
    if len(ss) and iso_mask.mean() >= 0.5:
        dt_iso = pd.to_datetime(ss.where(iso_mask), errors="coerce", dayfirst=False)
        ok = dt_iso.notna().mean()
        if ok >= 0.5:
            has_time = (dt_iso.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (dt_iso, "DATETIME" if has_time else "DATE")

    # 5) Excel serial -> datetime/date (seulement si nom datey)
    if colname_has(DATE_NAME_HINTS, name):
        as_num = pd.to_numeric(ss, errors="coerce")
        safe_mask = (as_num >= 60) & (as_num <= 60000)
        if (as_num.notna().mean() if len(ss) else 0.0) >= 0.9 and safe_mask.mean() >= 0.7:
            parsed = excel_serial_to_datetime(as_num)
            ok = parsed.notna().mean() if len(ss) else 0.0
            if ok >= 0.7:
                has_time = (parsed.dt.time.astype(str) != "00:00:00").mean() > 0.2
                return (parsed, "DATETIME" if has_time else "DATE")

    # 6) Tokens de date
    date_token_ratio = ss.fillna("").str.contains(DATE_TOKEN_RE, na=False).mean()
    if colname_has(DATE_NAME_HINTS, name) or (date_token_ratio >= 0.30):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
            dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
            dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
        dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1
        ok = dt.notna().mean() if len(ss) else 0.0
        ok_thresh = 0.50 if colname_has(DATE_NAME_HINTS, name) else 0.70
        if ok >= ok_thresh:
            has_time = (dt.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (dt, "DATETIME" if has_time else "DATE")

    # 7) Bool strict
    mb = ss.str.lower().map(lambda x: True if x in MAP_TRUE else (False if x in MAP_FALSE else pd.NA))
    if len(ss) and mb.notna().mean() >= 0.98:
        return mb.astype("boolean"), "BOOL"

    # 8) Nombre générique
    nums = parse_number_like(ss)
    if (nums.notna().mean() if len(ss) else 0) >= 0.85:
        nz = nums.dropna()
        if len(nz) and (np.mod(nz, 1) == 0).all():
            return nums.astype("Int64"), "INT"
        return nums.astype("Float64"), "FLOAT"

    # 9) Texte
    return ss.astype("string"), "STRING"

def sa_type(tag: str):
    return {
        "DATE": satypes.Date(),
        "DATETIME": satypes.DateTime(),    # timestamp without time zone
        "INT": satypes.Integer(),
        "FLOAT": satypes.Float(precision=53),
        "BOOL": satypes.Boolean(),
        "STRING": satypes.Text()
    }.get(tag, satypes.Text())

def prepare_for_sql(df: pd.DataFrame, schema_tags: dict):
    out = df.copy()
    for c, tag in schema_tags.items():
        if tag == "DATE":
            out[c] = pd.to_datetime(out[c], errors="coerce").dt.date
        elif tag == "DATETIME":
            out[c] = pd.to_datetime(out[c], errors="coerce")
    dtype_map = {c: sa_type(tag) for c, tag in schema_tags.items()}
    return out, dtype_map

def pg_ident(name: str, maxlen=63) -> str:
    return name[:maxlen]

# ============== MAIN ==============
def main():
    xlsx = Path(FILE_PATH)
    log(f"Fichier: {xlsx}")
    if not xlsx.exists():
        log("[error] Fichier introuvable"); return

    # Connexion DB
    try:
        engine = get_engine()
        with engine.connect() as conn:
            ver = conn.execute(text("select version();")).scalar()
            log("[ok] Connecté PostgreSQL")
            log(ver)
    except (OperationalError, RuntimeError) as e:
        log(f"[error] Connexion PostgreSQL échouée: {e}")
        return

    try:
        sheets = pd.read_excel(xlsx, sheet_name=None, engine="openpyxl", header=None)
    except Exception as e:
        log(f"[error] Lecture échouée: {e}"); return

    file_base = snake_id(xlsx.stem)
    done = 0

    for name, raw in sheets.items():
        log(f"\n[feuille] {name}: shape initiale={raw.shape}")

        raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)
        log(f"[feuille] {name}: après drop vides -> {raw.shape}")
        if raw.empty:
            log(f"[feuille] {name}: vide, on passe.")
            continue

        h = choose_header_row(raw, scan=30)
        log(f"[feuille] {name}: header choisi à la ligne {h}")

        # Schéma STRICT du fichier
        df_src = raw.iloc[h+1:].copy()
        src_cols_original = raw.iloc[h].tolist()
        df_src.columns = src_cols_original
        df_src = df_src.dropna(how="all").copy()
        df_src = drop_empty_columns(df_src)
        log(f"[feuille] {name}: shape après affectation des en-têtes -> {df_src.shape}")
        if df_src.empty:
            log(f"[feuille] {name}: vide après normalisation, on passe.")
            continue

        # noms de colonnes Postgres (snake_case + unicité) — aucune colonne ajoutée
        pg_cols = make_unique_columns([snake_id(c) for c in df_src.columns])

        # inférence sur colonnes d'origine -> vrais dtypes pandas
        schema_src = {}
        casted_df = df_src.copy()
        for c in list(df_src.columns):
            casted, tag = infer_col(df_src[c])
            casted_df[c] = casted
            schema_src[c] = tag

        # Affichage
        table_base = file_base if len(sheets) == 1 else f"{file_base}__{snake_id(name)}"
        table_name = pg_ident(table_base)

        print(f"\n--- Feuille: {name} -> {TARGET_SCHEMA}.{table_name} (SCHÉMA SOURCE STRICT / PostgreSQL) ---")
        print(f"rows={len(casted_df)} | cols={casted_df.shape[1]}")
        print("\nMapping colonnes (Excel → nom_pg) :")
        for src_name, pg_name in zip(df_src.columns, pg_cols):
            print(f"  - {src_name}  ->  {pg_name}")
        print("\nSchema détecté (types PostgreSQL) :")
        for src_name, pg_name in zip(df_src.columns, pg_cols):
            tag = schema_src.get(src_name, "STRING")
            print(f"  - {pg_name}: {sa_type(tag).__class__.__name__}  (source: {src_name}, tag: {tag})")

        if SHOW_SAMPLE:
            with pd.option_context("display.max_columns", 220, "display.width", 260):
                print("\nSample (top 10) :")
                print(casted_df.head(10))

        # ===== INSERTION DB =====
        try:
            df_for_sql = casted_df.copy()
            df_for_sql.columns = pg_cols
            schema_sql = {pg: schema_src[src] for src, pg in zip(df_src.columns, pg_cols)}
            df_sql, dtype_map = prepare_for_sql(df_for_sql, schema_sql)

            df_sql.to_sql(
                name=table_name,
                con=engine,
                schema=TARGET_SCHEMA,
                if_exists=IF_EXISTS_MODE,
                index=False,
                dtype=dtype_map,
                method="multi",
                chunksize=1000
            )
            log(f"[ok] Inserted into {TARGET_SCHEMA}.{table_name}")
            done += 1
        except Exception as e:
            log(f"[error] Échec insertion {TARGET_SCHEMA}.{table_name}: {e}")

    print(f"\nDone. Feuilles analysées & importées: {done}")

if __name__ == "__main__":
    main()


[15:59:09] Fichier: C:\globasoft\aerotech\fic\Suivi Projets AEC - source.xlsx
[15:59:09] [ok] Connecté PostgreSQL
[15:59:09] PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit
[15:59:10] 
[feuille] 2025mai: shape initiale=(226, 82)
[15:59:10] [feuille] 2025mai: après drop vides -> (226, 82)
[15:59:10] [feuille] 2025mai: header choisi à la ligne 3
[15:59:10] [feuille] 2025mai: shape après affectation des en-têtes -> (222, 75)

--- Feuille: 2025mai -> public.suivi_projets_aec_source (SCHÉMA SOURCE STRICT / PostgreSQL) ---
rows=222 | cols=75

Mapping colonnes (Excel → nom_pg) :
  - Comments  ->  comments
  - REVUE RAF  ->  revue_raf
  - GTM  ->  gtm
  - Statut suivi de projets  ->  statut_suivi_de_projets
  - Top custo  ->  top_custo
  - Num Projets  ->  num_projets
  - Réunion RAF  ->  reunion_raf
  - CdP  ->  cdp
  - Date fin  ->  date_fin
  - Total Budget
€  ->  total_budget
  - Ecart Budget M-1
€  ->  ecart_budget